In [33]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
from typing import List, Type, Optional
from pydantic import BaseModel, create_model, field_validator, Field, ConfigDict
from langchain_core.tools import BaseTool, InjectedToolArg
from langchain_core.callbacks import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticToolsParser

OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [37]:
from typing import Annotated
import env
import utils
import importlib 
importlib.reload(env)
importlib.reload(utils)
from pathlib import Path

setup_data_path = Path().resolve() / 'setup_data'

envs = utils.load_envs(setup_data_path, 'Version_0')

agents = ['Alice', 'Bob', 'Chad', 'Dave']
class Point3D(BaseModel):
    x: int = Field('X Coordinate')
    y: int = Field('Y Coordinate')
    z: int = Field('Z Coordinate')

# Define a validator to ensure each point is a valid 3D point
    @field_validator('x', 'y', 'z')
    def check_coordinates(cls, v):
        if not isinstance(v, int):
            raise ValueError('Coordinate must be an integer')
        return v

dict_schema = {'interface' : {agent : (List[Point3D], Field(f'A path of 3D points for Agent {agent}, where the list of points is an array and each point is a dictionary.')) for agent in agents} | {'env' : (Annotated[env.env, InjectedToolArg], Field('env object'))}}

Interface = create_model('interface', **dict_schema['interface'], __config__=ConfigDict(arbitrary_types_allowed=True))




test_inp = {
    'Alice': [{'x': 1, 'y': 2, 'z': 3}, {'x': 4, 'y': 5, 'z': 6}],
    'Bob': [{'x': 7, 'y': 8, 'z': 9}],
    'Chad': [{'x': 10, 'y': 11, 'z': 12}],
    'Dave': [{'x': 13, 'y': 14, 'z': 15}],
    'env' : envs[0],
}

Interface.validate(test_inp)

interface(Alice=[Point3D(x=1, y=2, z=3), Point3D(x=4, y=5, z=6)], Bob=[Point3D(x=7, y=8, z=9)], Chad=[Point3D(x=10, y=11, z=12)], Dave=[Point3D(x=13, y=14, z=15)], env=<env.env object at 0x1372a1be0>)

In [16]:
class AgentsInterface(BaseTool):
    name: str = "AgentsInterface"
    description: str = "How you instruct the agents to follow a particular path. Each agent should recieve a path according to the argument description."
    args_schema: Type[BaseModel] = Interface
    return_direct: bool = True
        
        
    def _run(
        self, **kwargs
    ) -> str:
        """Use the tool."""
        # The "$xtra" prefix on a kwarg means it isn't an agent path and serves another purpose
        # Ok i was wrong about the $xtra stuff bc it would have to go through the pydantic interface which would make it an arg for the LLM
        # What i will do it somehow set the env value in the state, and then pass the state to the tool
        # IDk man check this out https://langchain-ai.github.io/langgraph/how-tos/pass-run-time-values-to-tools/#define-the-tools
        # Might not work, good luck!!
        print(kwargs)
        agent_paths = {k : [(p.x, p.y, p.z)for p in v] for k, v in kwargs.items()} # Converts point object to tuple
        print(agent_paths)
        return 'Good job!'

In [17]:
# Toy Experiment Env
from pathlib import Path
import env
import utils

setup_data_path = Path().resolve() / 'setup_data'
envs = utils.load_envs(setup_data_path, 'Version_0')

In [18]:
llm = ChatOpenAI(model='gpt-4o-mini')

tools = [AgentsInterface()]

llm_with_tools = llm.bind_tools(tools)

In [43]:
query = '''Plan paths for agents to navigate a 3D grid to reach their respective goals and avoid collision.
You are given:
1) a list of obstacle coordinates (x, y, z): locations of the obstacle grid cells, agents must avoid them.
2) a list of [([name], [init], [goal]) tuples], [init] and [goal] are 3D coordinates of the initial position and goal position of agent named [name].
3) a previous plan, if any, and why it failed. Analyze this information and re-plan a collision-free path.

How to plan a <path>:
1) Make sure each path does not touch any obstacle or another agent.
2) Create a set of points for each agent to go from their init coordinates to their goal coordinates.
3) Make sure the coordinates are exactly one step away from each other, in one direction. Note - you may only move in one direction at a time.
Example of a <path>: [{'x' : 1, 'y' : 1, 'z' : 1}, {'x' : 1, 'y' : 1, 'z' : 2}, {'x' : 1, 'y' : 1, 'z' : 3},...]

Output Instruction: Use the Agent Interface tool provided to output your final plan for the agents. 

At the current step: Grid size: 10 x 10 x 10
Agents: [Alice, Bob, Chad, Dave]
Obstacles: (5, 9, 10) (10, 3, 10) (4, 7, 9) (7, 6, 2) (9, 3, 9) (10, 7, 9) (5, 7, 7) (7, 3, 9) (6, 3, 10) (4, 8, 8)
Agent Alice init: (9, 3, 2) goal: (6, 1, 3)
Agent Bob init: (2, 6, 4) goal: (4, 3, 9)
Agent Chad init: (1, 10, 3) goal: (3, 1, 5)
Agent Dave init: (1, 4, 2) goal: (6, 8, 9)
Your reasoning and plan is'''

In [44]:
chain = llm_with_tools | PydanticToolsParser(tools = [AgentsInterface])

In [58]:
out = llm_with_tools.invoke(query)

AttributeError: 'str' object has no attribute 'items'

In [48]:
PydanticToolsParser(tools = [AgentsInterface]).invoke(out)[0]

__main__.AgentsInterface

In [75]:
call = out.tool_calls[0]

call['args'].update({'$xtra_env' : 'env43'})
print(call['args'])
AgentsInterface().invoke(call)

{'Alice': [{'x': 9, 'y': 3, 'z': 2}, {'x': 8, 'y': 3, 'z': 2}, {'x': 7, 'y': 3, 'z': 2}, {'x': 6, 'y': 3, 'z': 2}, {'x': 6, 'y': 2, 'z': 2}, {'x': 6, 'y': 1, 'z': 2}, {'x': 6, 'y': 1, 'z': 3}], 'Bob': [{'x': 2, 'y': 6, 'z': 4}, {'x': 3, 'y': 6, 'z': 4}, {'x': 4, 'y': 6, 'z': 4}, {'x': 4, 'y': 5, 'z': 4}, {'x': 4, 'y': 4, 'z': 4}, {'x': 4, 'y': 3, 'z': 4}, {'x': 4, 'y': 3, 'z': 5}, {'x': 4, 'y': 3, 'z': 6}, {'x': 4, 'y': 3, 'z': 7}, {'x': 4, 'y': 3, 'z': 8}, {'x': 4, 'y': 3, 'z': 9}], 'Chad': [{'x': 1, 'y': 10, 'z': 3}, {'x': 1, 'y': 9, 'z': 3}, {'x': 1, 'y': 8, 'z': 3}, {'x': 1, 'y': 7, 'z': 3}, {'x': 1, 'y': 6, 'z': 3}, {'x': 2, 'y': 5, 'z': 3}, {'x': 3, 'y': 5, 'z': 3}, {'x': 3, 'y': 4, 'z': 3}, {'x': 3, 'y': 3, 'z': 3}, {'x': 3, 'y': 2, 'z': 3}, {'x': 3, 'y': 1, 'z': 3}, {'x': 3, 'y': 1, 'z': 4}, {'x': 3, 'y': 1, 'z': 5}], 'Dave': [{'x': 1, 'y': 4, 'z': 2}, {'x': 1, 'y': 5, 'z': 2}, {'x': 1, 'y': 6, 'z': 2}, {'x': 1, 'y': 7, 'z': 2}, {'x': 1, 'y': 8, 'z': 2}, {'x': 1, 'y': 9, 'z': 2

ToolMessage(content='Good job!', name='AgentsInterface', tool_call_id='call_dRc0I7kvjxdoDwcaMXFU12Bf')

In [16]:
from langchain_core.messages import AIMessage
t = AIMessage(content='To plan collision-free paths for the agents, we will follow these steps:\n\n1. **Identify the Obstacles and Agent Positions**: \n   - Obstacles: (5, 9, 10), (10, 3, 10), (4, 7, 9), (7, 6, 2), (9, 3, 9), (10, 7, 9), (5, 7, 7), (7, 3, 9), (6, 3, 10), (4, 8, 8)\n   - Agent Positions:\n     - Alice: Initial (9, 3, 2) → Goal (6, 1, 3)\n     - Bob: Initial (2, 6, 4) → Goal (4, 3, 9)\n     - Chad: Initial (1, 10, 3) → Goal (3, 1, 5)\n     - Dave: Initial (1, 4, 2) → Goal (6, 8, 9)\n\n2. **Analyze Possible Paths**:\n   - Each agent must navigate from their initial position to their goal while avoiding obstacles and other agents.\n   - The path must be made up of discrete steps, moving one unit in one direction at a time.\n\n3. **Plan the Paths**:\n   - **Alice**: Needs to move from (9, 3, 2) to (6, 1, 3). A possible path can be:\n     - Move to (8, 3, 2)\n     - Move to (7, 3, 2)\n     - Move to (6, 3, 2)\n     - Move to (6, 2, 2)\n     - Move to (6, 1, 2)\n     - Move to (6, 1, 3)\n   - **Bob**: Needs to move from (2, 6, 4) to (4, 3, 9). A possible path can be:\n     - Move to (2, 6, 5)\n     - Move to (2, 6, 6)\n     - Move to (2, 6, 7)\n     - Move to (2, 6, 8)\n     - Move to (2, 6, 9)\n     - Move to (2, 5, 9)\n     - Move to (3, 5, 9)\n     - Move to (4, 5, 9)\n     - Move to (4, 4, 9)\n     - Move to (4, 3, 9)\n   - **Chad**: Needs to move from (1, 10, 3) to (3, 1, 5). A possible path can be:\n     - Move to (1, 9, 3)\n     - Move to (1, 8, 3)\n     - Move to (1, 7, 3)\n     - Move to (1, 6, 3)\n     - Move to (1, 5, 3)\n     - Move to (1, 4, 3)\n     - Move to (1, 3, 3)\n     - Move to (2, 3, 3)\n     - Move to (2, 2, 3)\n     - Move to (3, 2, 3)\n     - Move to (3, 1, 3)\n     - Move to (3, 1, 4)\n     - Move to (3, 1, 5)\n   - **Dave**: Needs to move from (1, 4, 2) to (6, 8, 9). A possible path can be:\n     - Move to (1, 4, 3)\n     - Move to (1, 4, 4)\n     - Move to (1, 4, 5)\n     - Move to (1, 4, 6)\n     - Move to (1, 4, 7)\n     - Move to (1, 4, 8)\n     - Move to (1, 4, 9)\n     - Move to (2, 4, 9)\n     - Move to (3, 4, 9)\n     - Move to (4, 4, 9)\n     - Move to (5, 4, 9)\n     - Move to (6, 4, 9)\n     - Move to (6, 5, 9)\n     - Move to (6, 6, 9)\n     - Move to (6, 7, 9)\n     - Move to (6, 8, 9)\n\n4. **Check for Collisions**:\n   - Ensure that the paths do not intersect with each other or pass through the obstacle coordinates.\n\n5. **Final Paths**:\nNow we will structure these paths according to the Agent Interface tool:\n\n```json\n{\n  "Alice": [\n    {"x": 9, "y": 3, "z": 2},\n    {"x": 8, "y": 3, "z": 2},\n    {"x": 7, "y": 3, "z": 2},\n    {"x": 6, "y": 3, "z": 2},\n    {"x": 6, "y": 2, "z": 2},\n    {"x": 6, "y": 1, "z": 2},\n    {"x": 6, "y": 1, "z": 3}\n  ],\n  "Bob": [\n    {"x": 2, "y": 6, "z": 4},\n    {"x": 2, "y": 6, "z": 5},\n    {"x": 2, "y": 6, "z": 6},\n    {"x": 2, "y": 6, "z": 7},\n    {"x": 2, "y": 6, "z": 8},\n    {"x": 2, "y": 6, "z": 9},\n    {"x": 2, "y": 5, "z": 9},\n    {"x": 3, "y": 5, "z": 9},\n    {"x": 4, "y": 5, "z": 9},\n    {"x": 4, "y": 4, "z": 9},\n    {"x": 4, "y": 3, "z": 9}\n  ],\n  "Chad": [\n    {"x": 1, "y": 10, "z": 3},\n    {"x": 1, "y": 9, "z": 3},\n    {"x": 1, "y": 8, "z": 3},\n    {"x": 1, "y": 7, "z": 3},\n    {"x": 1, "y": 6, "z": 3},\n    {"x": 1, "y": 5, "z": 3},\n    {"x": 1, "y": 4, "z": 3},\n    {"x": 1, "y": 3, "z": 3},\n    {"x": 2, "y": 3, "z": 3},\n    {"x": 2, "y": 2, "z": 3},\n    {"x": 3, "y": 2, "z": 3},\n    {"x": 3, "y": 1, "z": 3},\n    {"x": 3, "y": 1, "z": 4},\n    {"x": 3, "y": 1, "z": 5}\n  ],\n  "Dave": [\n    {"x": 1, "y": 4, "z": 2},\n    {"x": 1, "y": 4, "z": 3},\n    {"x": 1, "y": 4, "z": 4},\n    {"x": 1, "y": 4, "z": 5},\n    {"x": 1, "y": 4, "z": 6},\n    {"x": 1, "y": 4, "z": 7},\n    {"x": 1, "y": 4, "z": 8},\n    {"x": 2, "y": 4, "z": 9},\n    {"x": 3, "y": 4, "z": 9},\n    {"x": 4, "y": 4, "z": 9},\n    {"x": 5, "y": 4, "z": 9},\n    {"x": 6, "y": 4, "z": 9},\n    {"x": 6, "y": 5, "z": 9},\n    {"x": 6, "y": 6, "z": 9},\n    {"x": 6, "y": 7, "z": 9},\n    {"x": 6, "y": 8, "z": 9}\n  ]\n}\n```\n\nNow I\'ll pass this path information to the Agents Interface tool.', additional_kwargs={'tool_calls': [{'id': 'call_E3DQgIeJHJZvedYn2WOIiagi', 'function': {'arguments': '{"Alice":[{"x":9,"y":3,"z":2},{"x":8,"y":3,"z":2},{"x":7,"y":3,"z":2},{"x":6,"y":3,"z":2},{"x":6,"y":2,"z":2},{"x":6,"y":1,"z":2},{"x":6,"y":1,"z":3}],"Bob":[{"x":2,"y":6,"z":4},{"x":2,"y":6,"z":5},{"x":2,"y":6,"z":6},{"x":2,"y":6,"z":7},{"x":2,"y":6,"z":8},{"x":2,"y":6,"z":9},{"x":2,"y":5,"z":9},{"x":3,"y":5,"z":9},{"x":4,"y":5,"z":9},{"x":4,"y":4,"z":9},{"x":4,"y":3,"z":9}],"Chad":[{"x":1,"y":10,"z":3},{"x":1,"y":9,"z":3},{"x":1,"y":8,"z":3},{"x":1,"y":7,"z":3},{"x":1,"y":6,"z":3},{"x":1,"y":5,"z":3},{"x":1,"y":4,"z":3},{"x":1,"y":3,"z":3},{"x":2,"y":3,"z":3},{"x":2,"y":2,"z":3},{"x":3,"y":2,"z":3},{"x":3,"y":1,"z":3},{"x":3,"y":1,"z":4},{"x":3,"y":1,"z":5}],"Dave":[{"x":1,"y":4,"z":2},{"x":1,"y":4,"z":3},{"x":1,"y":4,"z":4},{"x":1,"y":4,"z":5},{"x":1,"y":4,"z":6},{"x":1,"y":4,"z":7},{"x":1,"y":4,"z":8},{"x":2,"y":4,"z":9},{"x":3,"y":4,"z":9},{"x":4,"y":4,"z":9},{"x":5,"y":4,"z":9},{"x":6,"y":4,"z":9},{"x":6,"y":5,"z":9},{"x":6,"y":6,"z":9},{"x":6,"y":7,"z":9},{"x":6,"y":8,"z":9}]}', 'name': 'AgentsInterface'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2626, 'prompt_tokens': 841, 'total_tokens': 3467, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-37363938-2cd1-4c56-b7bc-d4e1a571729c-0', tool_calls=[{'name': 'AgentsInterface', 'args': {'Alice': [{'x': 9, 'y': 3, 'z': 2}, {'x': 8, 'y': 3, 'z': 2}, {'x': 7, 'y': 3, 'z': 2}, {'x': 6, 'y': 3, 'z': 2}, {'x': 6, 'y': 2, 'z': 2}, {'x': 6, 'y': 1, 'z': 2}, {'x': 6, 'y': 1, 'z': 3}], 'Bob': [{'x': 2, 'y': 6, 'z': 4}, {'x': 2, 'y': 6, 'z': 5}, {'x': 2, 'y': 6, 'z': 6}, {'x': 2, 'y': 6, 'z': 7}, {'x': 2, 'y': 6, 'z': 8}, {'x': 2, 'y': 6, 'z': 9}, {'x': 2, 'y': 5, 'z': 9}, {'x': 3, 'y': 5, 'z': 9}, {'x': 4, 'y': 5, 'z': 9}, {'x': 4, 'y': 4, 'z': 9}, {'x': 4, 'y': 3, 'z': 9}], 'Chad': [{'x': 1, 'y': 10, 'z': 3}, {'x': 1, 'y': 9, 'z': 3}, {'x': 1, 'y': 8, 'z': 3}, {'x': 1, 'y': 7, 'z': 3}, {'x': 1, 'y': 6, 'z': 3}, {'x': 1, 'y': 5, 'z': 3}, {'x': 1, 'y': 4, 'z': 3}, {'x': 1, 'y': 3, 'z': 3}, {'x': 2, 'y': 3, 'z': 3}, {'x': 2, 'y': 2, 'z': 3}, {'x': 3, 'y': 2, 'z': 3}, {'x': 3, 'y': 1, 'z': 3}, {'x': 3, 'y': 1, 'z': 4}, {'x': 3, 'y': 1, 'z': 5}], 'Dave': [{'x': 1, 'y': 4, 'z': 2}, {'x': 1, 'y': 4, 'z': 3}, {'x': 1, 'y': 4, 'z': 4}, {'x': 1, 'y': 4, 'z': 5}, {'x': 1, 'y': 4, 'z': 6}, {'x': 1, 'y': 4, 'z': 7}, {'x': 1, 'y': 4, 'z': 8}, {'x': 2, 'y': 4, 'z': 9}, {'x': 3, 'y': 4, 'z': 9}, {'x': 4, 'y': 4, 'z': 9}, {'x': 5, 'y': 4, 'z': 9}, {'x': 6, 'y': 4, 'z': 9}, {'x': 6, 'y': 5, 'z': 9}, {'x': 6, 'y': 6, 'z': 9}, {'x': 6, 'y': 7, 'z': 9}, {'x': 6, 'y': 8, 'z': 9}]}, 'id': 'call_E3DQgIeJHJZvedYn2WOIiagi', 'type': 'tool_call'}], usage_metadata={'input_tokens': 841, 'output_tokens': 2626, 'total_tokens': 3467, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
env = AgentsInterface()

{'default': 'A path of 3D points for Agent Bob, where the list of points is an array and each point is a dictionary.',
 'items': {'$ref': '#/$defs/Point3D'},
 'title': 'Bob',
 'type': 'array'}

In [4]:
client = openai.OpenAI()

In [8]:
result = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a series of haikus about recursion in programming."
        }
    ]
)

In [13]:
print(result.usage.completion_tokens)

100
